In [1]:
# Importation des bases
import pandas as pd
import functions as fc
import numpy as np

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [2]:
df = fc.clean_data_etab(
    dict_data['etablissements']
    )

In [3]:
request_template = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{"multi_match" : {{"query":"{nom_etablissement}",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}}}},
        {{ "match": {{ "sir_adr_et_com_lib":  "{commune}" }}}}
      ],
      "filter": [
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''



In [4]:
out = fc.pipeline_request(df, request_template, cols = ['nom_etablissement',  'commune', 'code_apet'])

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


8963
Matched OK True     0.556175
False    0.443825
Name: match, dtype: float64


In [ ]:
out[["nom_etablissement","nom_etab_elastic", "textual_distance", "match"]].head(20)

In [ ]:
request_template_geo = '''{{
"query": {{ "bool": {{ "should": {{ "match": {{ "rs_denom":   "{nom_etablissement}" }} }}, "filter": [ {{ "geo_distance": {{ "distance": "0.5km", "location": {{ "lat": "{y}",  "lon": "{x}" }}  }} }}]   }}  }}, "size": 1
}}
'''

## Les émissions de CO2

On va commencer par se focaliser sur les gros émetteurs de CO2

In [ ]:
emissions = dict_data['emissions']
emissions_co2 = emissions.loc[emissions['polluant'].str.contains("CO2")]
emissions_co2 = emissions_co2.loc[emissions_co2['polluant'].str.contains('total')]
emissions_co2.head(1)

In [ ]:
emissions_co2['quantite'].div(1000).plot.hist(logx = True)

In [ ]:
dict_data.head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dict_data['etablissements']['identifiant'].nunique()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data